In [1]:
import numpy as np

def compute_rotation_matrix(phi, beta):
    """
    Compute the rotation matrix given phi and beta angles.
    
    Parameters:
    phi : float - Directional angle (radians)
    beta : float - Bending angle (radians)
    
    Returns:
    3x3 rotation matrix (numpy array)
    """
    a = np.cos(beta) * np.cos(phi)**2 + np.sin(phi)**2
    b = (-1 + np.cos(beta)) * np.cos(phi) * np.sin(phi)
    c = np.cos(phi) * np.sin(beta)
    d = np.sin(beta) * np.sin(phi)
    e = np.cos(beta)
    f = np.cos(beta) * np.sin(phi)**2 + np.cos(phi)**2

    return np.array([[a, b, c], [b, f, d], [-c, -d, e]])

def calc_displacement(rho, beta, phi):
    """
    Compute the displacement vector based on curvature parameters.
    
    Parameters:
    rho : float - Curvature
    beta : float - Bending angle (radians)
    phi : float - Directional angle (radians)
    
    Returns:
    3D displacement vector (numpy array)
    """
    return (1 / rho) * np.array([(1 - np.cos(beta)) * np.sin(phi),
                                 (1 - np.cos(beta)) * np.cos(phi),
                                 np.sin(beta)])

def get_beta(l1, l2, l3, r):
    """
    Compute the bending angle beta from segment lengths.
    
    Parameters:
    l1, l2, l3 : float - Three segment lengths
    r : float - Radius parameter
    
    Returns:
    Beta (float) - Bending angle in radians
    """
    return 2 * np.sqrt(l1**2 + l2**2 + l3**2 - l1*l2 - l1*l3 - l2*l3) / (3 * r)

def get_phi(l1, l2, l3):
    """
    Compute the directional angle phi from segment lengths.
    
    Parameters:
    l1, l2, l3 : float - Three segment lengths
    
    Returns:
    Phi (float) - Directional angle in radians
    """
    return np.arctan2(3 * (l2 - l3), np.sqrt(3) * (l2 + l3 - 2 * l1))

def length_to_position_single(l, rigid_len, r):
    """
    Compute the end-effector position for a single segment.
    
    Parameters:
    l : list - Three segment lengths [l1, l2, l3]
    rigid_len : float - Length of the rigid body
    r : float - Radius parameter
    
    Returns:
    3D end-effector position (numpy array)
    """
    # Compute parameters
    phi = get_phi(l[0], l[1], l[2])
    beta = get_beta(l[0], l[1], l[2], r)
    lc = np.mean(l)  # Average segment length
    rho = beta / lc  # Curvature

    # Compute displacement and rotation
    displacement = calc_displacement(rho, beta, phi)
    rotation = compute_rotation_matrix(phi, beta)
    direction = rotation @ np.array([0, 0, 1])  # Transform the direction

    # Compute final position
    end_position = displacement + direction * rigid_len
    #return end_position_final
    return end_position



In [2]:
import numpy as np
from scipy.optimize import fsolve, root

def constant_curvature(p):
    x = p[0]
    y = p[1]
    z = p[2]
    d = 19  # Distance from the center of the chamber to the center of the base platform (mm)
    aa = 0  # Fixed distance from the base to the cross-section of the chamber

    # Special case: If the position is at the origin (0, 0), return straight lengths
    if x == 0 and y == 0:
        q1 = (z - aa) 
        q2 = (z - aa) 
        q3 = (z - aa)
        return np.array([q1, q2, q3])

    # Calculate the angle in the XY plane
    phi = np.arctan2(y, x)
    # Initial guess for the solver
    initial_guess = [x, y, z, 1.0, 0.1]  # Adjusted initial guess

    # Define the nonlinear equations to solve
    def nonlinear_equations(variables):
        x0, y0, z0, k0, theta0 = variables
        f = np.zeros(5)
        f[0] = 2 * np.sqrt(x0**2 + y0**2) / (x0**2 + y0**2 + z0**2) - k0
        f[1] = np.arccos(np.clip(1 - k0 * np.sqrt(x0**2 + y0**2), -1, 1)) - theta0
        f[2] = x0 + aa * np.sin(theta0) * np.cos(phi) - x
        f[3] = y0 + aa * np.sin(theta0) * np.sin(phi) - y
        f[4] = z0 + aa * np.cos(theta0) - z
        return f

    # Solve the nonlinear equations using fsolve
    try:
        solution = fsolve(nonlinear_equations, initial_guess, xtol=1e-12, maxfev=int(1e8))
    except Exception as e:
        print("Fsolve failed with exception:", e)
        return np.array([np.nan, np.nan, np.nan])

    # Extract real parts of the solution
    x1, y1, z1, k, theta = solution

    # Calculate the radii of curvature for each chamber
    R1 = 1 / k - d * np.sin(phi)
    R2 = 1 / k + d * np.sin(np.pi / 3 + phi)
    R3 = 1 / k - d * np.cos(np.pi / 6 + phi)

    # Calculate the length changes for each chamber
    q1 = float(theta * R1)
    q2 = float(theta * R2)
    q3 = float(theta * R3)

    return np.array([q1, q2, q3])



In [2]:
# Example usage
l = [60, 80, 100]  # Example segment lengths
rigid_len = 0    # Rigid body length
r = 19 #60 / np.sqrt(3)  # Radius parameter

end_effector_pos = length_to_position_single(l, rigid_len, r)
end_effector_pos

array([-21.46018837,  37.17013659,  61.70658006])

In [12]:
# Example usage with arbitrary actuator lengths
x, y, z = -19.89288082,  16.47870796,  52.96450381  # Example lengths
p = [x,y,z]
final_length = constant_curvature(p)
final_length

array([50., 55., 78.])